<a href="https://colab.research.google.com/github/nada-mah/colabcollection/blob/main/smolvlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall transformers -y
!pip install git+https://github.com/huggingface/transformers accelerate

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6z6d0z2d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6z6d0z2d
  Resolved https://github.com/huggingface/transformers to commit 6daa3eeba582facb57cd71db8efb66998b12942f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
# It's highly recommanded to use `[decord]` feature for faster video loading.
!pip install qwen-vl-utils[decord]==0.0.8


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 42.2 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-3B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:


messages = [
    {
        "role": "user",
        "content": [
       {
                "type": "image",
                "image": "/content/Screenshot 2025-04-14 121406.png",
            },
            {"type": "text", "text": """You are an intelligent document parser tasked with extracting structured data from unstructured PDF content. Below is the context and requirement:

The documents are technical product PDFs that include a section often labeled "Ordering Logic", "Ordering Guide", "Ordering Information", or "Product Selector". This section contains key product details in a semi-structured or unstructured layout, including:

Attribute: A category or characteristic of the product (e.g., Size, Finish, Voltage).

Code: A short identifier or shorthand for a specific option (e.g., "120V" for Voltage).

Value: A description or meaning of the code (e.g., "120 Volts").

Your task is to extract each of these data points—Attribute, Code, and Value—and structure them into a standardized JSON format like this:
[
  {
    "attribute": "Voltage",
    "code": "120V",
    "value": "120 Volts"
  },
  {
    "attribute": "Finish",
    "code": "WH",
    "value": "White"
  }
]
Please interpret the given text, tables, or embedded data, even if formatting is inconsistent. Focus especially on the sections that resemble an Ordering Logic table."""}
        ]
    },
]


# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['```json\n[\n  {\n    "attribute": "System",\n    "code": "L",\n    "value": "L Track"\n  },\n  {\n    "attribute": "Model #",\n    "code": "LED9",\n    "value": "H Track"\n  },\n  {\n    "attribute": "Beam Spread",\n    "code": "S",\n    "value": "10°"\n  },\n  {\n    "attribute": "Color Temp",\n    "code": "27",\n    "value": "2700K"\n  },\n  {\n    "attribute": "Finish",\n    "']


In [ ]:


messages = [
    {
        "role": "user",
        "content": [
       {
                "type": "image",
                "image": "/content/wwww.png",
            },
            {"type": "text", "text": """Analyze the given image and extract details about the clothing worn by the person. For each clothing item, provide a concise, accurate description of the following attributes:
Clothing Type: Clearly state the type of clothing ,shape and sleeve in one item
Color: Identify the primary color and any secondary colors, patterns, or graphics on the clothing.
Ensure all descriptions are accurate and align with what is visually present in the image. The output should be formatted as a short, structured list for each clothing item."""}
        ]
    },
]


# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['- Clothing Type: T-shirt, short-sleeved, black\n  - Color: Black, with white graphic design on the chest and sleeves\n\n- Clothing Type: Hooded top, long-sleeved, black\n  - Color: Black, with a patterned design on the sleeves\n\n- Clothing Type: Jeans, straight-leg, pink\n  - Color: Pink, with a subtle pattern\n\n- Clothing Type: Sneakers, athletic, black and white\n  - Color: Black and white, with a classic design\n\n- Clothing Type: Hijab, head covering, black\n  - Color: Black, with a']


In [ ]:
# a = output_text[0].split('\n')

# a[0]

# smovl

In [ ]:
# import torch
# from PIL import Image
# from transformers import AutoProcessor, AutoModelForVision2Seq
# from transformers.image_utils import load_image

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# # Load images
# image = load_image("/content/bad.png")

# # Initialize processor and model
# processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
# model = AutoModelForVision2Seq.from_pretrained(
#     "HuggingFaceTB/SmolVLM-Instruct",
#     torch_dtype=torch.bfloat16,
# ).to(DEVICE)

# # Create input messages
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image"},
#             {"type": "text", "text": "Can you describe the outfit in the image? make sure your answer is short and focus only on the outfit"}
#         ]
#     },
# ]

# # Prepare inputs
# prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image], return_tensors="pt")
# inputs = inputs.to(DEVICE)

# # Generate outputs
# generated_ids = model.generate(**inputs, max_new_tokens=500)
# generated_texts = processor.batch_decode(
#     generated_ids,
#     skip_special_tokens=True,
# )

# print(generated_texts[0])

# deep

In [1]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL2.git
!cd DeepSeek-VL2 && pip install -e .

Cloning into 'DeepSeek-VL2'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 211 (delta 65), reused 41 (delta 41), pack-reused 122 (from 1)
Receiving objects: 100% (211/211), 31.04 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (95/95), done.
Obtaining file:///content/DeepSeek-VL2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer t

In [4]:
import sys
sys.path.append('/content/DeepSeek-VL2')

In [ ]:
!pip install numpy==1.26.4
exit() #rests runtime

In [ ]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images


# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-small"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

## single image conversation example
conversation = [
    {
        "role": "<|User|>",
        "content": "<image>\n<|ref|>the person<|/ref|>.",
        "images": ["/content/out_dc_upper (1).png"],
    },
    {"role": "<|Assistant|>", "content": ""},
]


# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


Python version is above 3.10, patching the collections module.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:100002
Add image token = ['<image>'] to the tokenizer
<image>:100003
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:100004
<|/ref|>:100005
<|det|>:100006
<|/det|>:100007
<|grounding|>:100008
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:100009
<|Assistant|>:100010



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM

# from deepseek_vl.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
# from deepseek_vl.utils.io import load_pil_images


# # specify the path to the model
# model_path = "deepseek-ai/deepseek-vl2-small"
# vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
# tokenizer = vl_chat_processor.tokenizer

# vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
# vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

# ## single image conversation example
# conversation = [
#     {
#         "role": "<|User|>",
#         "content": "<image>\n<|ref|>The giraffe at the back.<|/ref|>.",
#         "images": ["/content/out_dc_upper (1).png"],
#     },
#     {"role": "<|Assistant|>", "content": ""},
# ]

# ## multiple images (or in-context learning) conversation example
# # conversation = [
# #     {
# #         "role": "User",
# #         "content": "<image_placeholder>A dog wearing nothing in the foreground, "
# #                    "<image_placeholder>a dog wearing a santa hat, "
# #                    "<image_placeholder>a dog wearing a wizard outfit, and "
# #                    "<image_placeholder>what's the dog wearing?",
# #         "images": [
# #             "images/dog_a.png",
# #             "images/dog_b.png",
# #             "images/dog_c.png",
# #             "images/dog_d.png",
# #         ],
# #     },
# #     {"role": "Assistant", "content": ""}
# # ]

# # load images and prepare for inputs
# pil_images = load_pil_images(conversation)
# prepare_inputs = vl_chat_processor(
#     conversations=conversation,
#     images=pil_images,
#     force_batchify=True,
#     system_prompt=""
# ).to(vl_gpt.device)

# # run image encoder to get the image embeddings
# inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
# outputs = vl_gpt.language_model.generate(
#     inputs_embeds=inputs_embeds,
#     attention_mask=prepare_inputs.attention_mask,
#     pad_token_id=tokenizer.eos_token_id,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=512,
#     do_sample=False,
#     use_cache=True
# )

# answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
# print(f"{prepare_inputs['sft_format'][0]}", answer)
